# [모듈 2.1] CodeBuild 프로젝트 생성

아래의 개발자 가이드 및 관련 명령어를 참고 해주세요.
- 개발자 가이드
    - [Create a build project (AWS CLI)](https://docs.aws.amazon.com/codebuild/latest/userguide/create-project-cli.html)
- 관련 명령어
    - aws codebuild list-projects --sort-by NAME --sort-order ASCENDING    
    - aws codebuild batch-get-projects --names iris-build-project    

# 1. 환경 설정


## 1.1. 파라미터 설정

In [1]:
import sagemaker
import boto3

region = sagemaker.Session().boto_region_name

account_id = boto3.client('sts').get_caller_identity().get('Account')


print("region: ", region)
print("account_id: ", account_id)

region:  ap-northeast-2
account_id:  057716757052


In [2]:
isDefaultBucket = True

if isDefaultBucket:

    bucket = sagemaker.Session().default_bucket()
else:
    bucket = "<Type your bucket name>"
    
print("Bucket: ", bucket)    


Bucket:  sagemaker-ap-northeast-2-057716757052


## 1.2.  Code Build 역할 ARN 가져오기 
- 워크샵의 셋업 단계에서 CodeBuild_Role: 은 아래와 미리 생성이 됨
    -  "arn:aws:iam::XXXXXX:role/MLOps-CodeBuildRole"

In [6]:
build_service_role = boto3.client('iam').get_role(RoleName = 'MLOps-CodeBuildRole').get('Role').get('Arn')
print("build_service_role: \n", build_service_role)



build_service_role: 
 arn:aws:iam::057716757052:role/MLOps-CodeBuildRole


## 1.3. Code Build Project 이름 설정

In [5]:
hello_code_build_project_name = "Hello_Code_Build"


아래와 같이 IAM 콘솔에 가셔서 왼쪽의 Roles(역할) 클릭하시고, sagemaker-notebook-CodeBuildRole-XXXXX 을 찾으신 후에 ARN 을 복사하시면 됩니다.

# 2. 빌드 프로젝트  JSON 정의

## 2.1. build_dic Json 정의

Code Build 프로젝트를 생성시에 정의를 아래의 'build_dic"  JSON 으로 정의해서 "build project" 를 생성 합니다. 결론적으로 정의된 도커 컨테이너가 buildspec.yml 에 정의된 "buildspec": "buildspec.yml" 를 최종 실행하게 됩니다.

```json
build_dic = {
  "name": hello_code_build_project_name,
  "description": "Hello Build Project",
  "source": {
    "type": "CODEPIPELINE",
    "buildspec": "buildspec.yml",
  },

```

"buildspec.yml" 는 이전 노트북에서 생성한 hello-code-repo 에 있습니다. 아래 내용을 보면 간단하게 `echo "dummy"` 를 출력을 하게 됩니다.  보통 "buildspec.yml"의 정의를 일반적으로 Python 파일을 실행 명령어 및 이를 위한 Python Package 설치 등으로 구성이 됩니다.

![hello_buildspec_yml.png](img/hello_buildspec_yml.png)

## 2.2. 환경 변수 정의
Code Build Project 을 생성시에 아래와 같이 환경 변수를 정의할 수 있습니다. 이러한 환경 변순는 buildspec_yml 파일 안에서 참조하여 사용할 수 있습니다. 예를 들어서 AWS_DEFAULT_REGION 변수에 리젼을 할당하여 저장할 수도 있고, 버킷 이름을 TEMPLATE_BUCKET 변수에 저장하여 사용할 수 있습니다.

```JSON
        "environment": {
            "type": "LINUX_CONTAINER",
            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
            "computeType": "BUILD_GENERAL1_SMALL",
            "environmentVariables": [
                {
                    "name": "IMAGE_REPO_NAME",
                    "value": "hello-model",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "IMAGE_TAG",
                    "value": "latest",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "AWS_ACCOUNT_ID",
                    "value": "057716757052",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "AWS_DEFAULT_REGION",
                    "value": "us-east-1",
                    "type": "PLAINTEXT"
                },
```

In [7]:
build_dic = {
  "name": hello_code_build_project_name,
  "description": "Hello Build Project",
  "source": {
    "type": "CODEPIPELINE",
    "buildspec": "buildspec.yml",
  },
  "artifacts": {
    "type": "CODEPIPELINE",
    "name": hello_code_build_project_name,
  },
  "cache": {
    "type": "NO_CACHE",
  },
  "environment": {
            "type": "LINUX_CONTAINER",
            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
            "computeType": "BUILD_GENERAL1_SMALL",
            "environmentVariables": [
                {
                    "name": "IMAGE_REPO_NAME",
                    "value": "hello-model",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "IMAGE_TAG",
                    "value": "latest",
                    "type": "PLAINTEXT"
                },
                {
                    "name": "AWS_ACCOUNT_ID",
                    "value": account_id,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "AWS_DEFAULT_REGION",
                    "value": region,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "TEMPLATE_BUCKET",
                    "value": bucket,
                    "type": "PLAINTEXT"
                },
                {
                    "name": "TEMPLATE_PREFIX",
                    "value": "codebuild",
                    "type": "PLAINTEXT"
                }
            ],
            "privilegedMode": False,
            "imagePullCredentialsType": "CODEBUILD"
        },
  "serviceRole": build_service_role,
  "timeoutInMinutes": 60,
  "queuedTimeoutInMinutes": 480,
  "badgeEnabled": False,
  "logsConfig": {
    "cloudWatchLogs": {
      "status": "ENABLED",
    },
    "s3Logs": {
      "status": "DISABLED",
      "encryptionDisabled": False
    }
  },
}

## 2.1. Dic 포맷을 JSON 으로 변경

In [8]:
import json

build_json = json.dumps(build_dic)
json.loads(build_json)

{'name': 'Hello_Code_Build',
 'description': 'Hello Build Project',
 'source': {'type': 'CODEPIPELINE', 'buildspec': 'buildspec.yml'},
 'artifacts': {'type': 'CODEPIPELINE', 'name': 'Hello_Code_Build'},
 'cache': {'type': 'NO_CACHE'},
 'environment': {'type': 'LINUX_CONTAINER',
  'image': 'aws/codebuild/amazonlinux2-x86_64-standard:3.0',
  'computeType': 'BUILD_GENERAL1_SMALL',
  'environmentVariables': [{'name': 'IMAGE_REPO_NAME',
    'value': 'hello-model',
    'type': 'PLAINTEXT'},
   {'name': 'IMAGE_TAG', 'value': 'latest', 'type': 'PLAINTEXT'},
   {'name': 'AWS_ACCOUNT_ID', 'value': '057716757052', 'type': 'PLAINTEXT'},
   {'name': 'AWS_DEFAULT_REGION',
    'value': 'ap-northeast-2',
    'type': 'PLAINTEXT'},
   {'name': 'TEMPLATE_BUCKET',
    'value': 'sagemaker-ap-northeast-2-057716757052',
    'type': 'PLAINTEXT'},
   {'name': 'TEMPLATE_PREFIX', 'value': 'codebuild', 'type': 'PLAINTEXT'}],
  'privilegedMode': False,
  'imagePullCredentialsType': 'CODEBUILD'},
 'serviceRole': 'a

## 2.2. JSON 파일 저장

In [9]:
import os

os.makedirs('src',exist_ok=True)

In [10]:
json_file_path = 'src/hello_build.json'
with open(json_file_path, "w") as outfile:
    outfile.write(build_json)

In [11]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

{'name': 'Hello_Code_Build',
 'description': 'Hello Build Project',
 'source': {'type': 'CODEPIPELINE', 'buildspec': 'buildspec.yml'},
 'artifacts': {'type': 'CODEPIPELINE', 'name': 'Hello_Code_Build'},
 'cache': {'type': 'NO_CACHE'},
 'environment': {'type': 'LINUX_CONTAINER',
  'image': 'aws/codebuild/amazonlinux2-x86_64-standard:3.0',
  'computeType': 'BUILD_GENERAL1_SMALL',
  'environmentVariables': [{'name': 'IMAGE_REPO_NAME',
    'value': 'hello-model',
    'type': 'PLAINTEXT'},
   {'name': 'IMAGE_TAG', 'value': 'latest', 'type': 'PLAINTEXT'},
   {'name': 'AWS_ACCOUNT_ID', 'value': '057716757052', 'type': 'PLAINTEXT'},
   {'name': 'AWS_DEFAULT_REGION',
    'value': 'ap-northeast-2',
    'type': 'PLAINTEXT'},
   {'name': 'TEMPLATE_BUCKET',
    'value': 'sagemaker-ap-northeast-2-057716757052',
    'type': 'PLAINTEXT'},
   {'name': 'TEMPLATE_PREFIX', 'value': 'codebuild', 'type': 'PLAINTEXT'}],
  'privilegedMode': False,
  'imagePullCredentialsType': 'CODEBUILD'},
 'serviceRole': 'a

# 3. 빌드 프로젝트 생성

In [12]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
aws codebuild create-project --cli-input-json file://$json_file_path

src/hello_build.json
{
    "project": {
        "name": "Hello_Code_Build",
        "arn": "arn:aws:codebuild:ap-northeast-2:057716757052:project/Hello_Code_Build",
        "description": "Hello Build Project",
        "source": {
            "type": "CODEPIPELINE",
            "buildspec": "buildspec.yml",
            "insecureSsl": false
        },
        "artifacts": {
            "type": "CODEPIPELINE",
            "name": "Hello_Code_Build",
            "packaging": "NONE",
            "encryptionDisabled": false
        },
        "cache": {
            "type": "NO_CACHE"
        },
        "environment": {
            "type": "LINUX_CONTAINER",
            "image": "aws/codebuild/amazonlinux2-x86_64-standard:3.0",
            "computeType": "BUILD_GENERAL1_SMALL",
            "environmentVariables": [
                {
                    "name": "IMAGE_REPO_NAME",
                    "value": "hello-model",
                    "type": "PLAINTEXT"
                },
           

## 참고: 코드 빌드 프로젝트 리스트
- [Boto3 ListProjects](https://docs.aws.amazon.com/codebuild/latest/APIReference/API_ListProjects.html)

In [13]:
%%sh 
aws codebuild list-projects --sort-by LAST_MODIFIED_TIME --sort-order DESCENDING

{
    "projects": [
        "Hello_Code_Build",
        "sagemaker-only-deploy-template-p-fsvuvjxvs1e3-modeldeploy",
        "sagemaker-only-deploy-template-p-fsvuvjxvs1e3-testing",
        "sagemaker-demo-project-gsmoon-p-9mvtktb4j47f-modelbuild",
        "sagemaker-demo-project-gsmoon-p-9mvtktb4j47f-modeldeploy",
        "sagemaker-demo-project-gsmoon-p-9mvtktb4j47f-testing",
        "sagemaker-scratch3-default-template-p-nszkgtfiasvk-modelbuild",
        "sagemaker-scratch3-default-template-p-nszkgtfiasvk-modeldeploy",
        "sagemaker-scratch3-default-template-p-nszkgtfiasvk-testing",
        "sagemaker-scratch2-default-template-p-t2zokhnjqc6z-modelbuild",
        "sagemaker-scratch2-default-template-p-t2zokhnjqc6z-modeldeploy",
        "sagemaker-scratch2-default-template-p-t2zokhnjqc6z-testing"
    ]
}


# 4. 프로젝트 이름 저장

In [14]:
%store hello_code_build_project_name
%store account_id
%store region
%store bucket


Stored 'hello_code_build_project_name' (str)
Stored 'account_id' (str)
Stored 'region' (str)
Stored 'bucket' (str)


# 5. 생성된 Hello_Code_Build 프로젝트

![code_build_project.png](img/code_build_project.png)